# Computer vision with neural networks

__Pré-requis : exécuter les cellules du fichier install.ipynb__

---

#### L'objectif est de construire et tester différents modèles de réseaux de neurones pour faire de la reconnaissance d'images, tout en prenant conscience de leurs limites.  
#### Les parties 1 à 4 seront consacrées à la reconnaissance de chiffres manuscrits, tandis que la dernière partie sera consacrée à l'utilisation d'un algorithme pré-entraîné pour faire de la détection d'objets.  
- #### Dans la partie 1 : nous travaillerons avec les données du problème de reconnaissance de chiffres manuscrits pour en comprendre le format et en évaluer la qualité
- #### Dans la partie 2 : nous construirons un réseau de neurones artificiel pour reconnaître les chiffres manuscrits
- #### Dans la partie 3 : nous construirons un réseau de neurones convolutionnel pour reconnaître les chiffres manuscrits
- #### Dans la partie 4 : nous porterons un regard critique sur les résultats des réseaux de neurones développés
- #### Dans la partie 5 : nous testerons un réseau de neurones pré-rentraîné bien plus puissant pour détecter des objets sur des images

----

# <font color="gray">Partie 1 - Comprendre les données utilisées</font>

On importe les librairies qui nous seront utiles pour cette partie :  
- __MNIST__ : pour l'import de la base de données MNIST contenant 70 000 chiffres manuscrits  
- __Numpy__ : pour travailler facilement des tableaux / matrices de données  
- __Matplotlib__ : pour faire des graphiques    

In [ ]:
from mnist import MNIST
import numpy as np
import matplotlib.pyplot as plt

On importe les données du problème composées d'__images__ (img) de chiffres manuscrits et de __labels__ (lbl) indiquant le chiffre (compris entre 0 et 9) représenté sur l'image.  

In [ ]:
# Import des données brutes
mndata = MNIST('../Resources/data')
img_train_src, lbl_train_src = mndata.load_training()
img_test_src, lbl_test_src = mndata.load_testing()

# Reformattage des données importées sous forme de tableaux aux bonnes dimensions
img_train = np.expand_dims(np.array([np.array(image).reshape(28,28) for image in img_train_src]),3)
img_test = np.expand_dims(np.array([np.array(image).reshape(28,28) for image in img_test_src]),3)
lbl_train = np.array(list(lbl_train_src))
lbl_test = np.array(list(lbl_test_src))

On a construit 4 jeux de données :
- __img_train__ : images qui seront utilisées pour entraîner l'IA - _dimensions : (60000,28,28,1_)
- __lbl_train__ : chiffres représentés sur les images de img_train - _dimensions : (60000,)_ (NB : le ième label correspond au label de la ième image)
- __img_test__ : images qui seront utilisées pour tester l'IA - _dimensions : (10000,28,28,1)_
- __lbl_test__ : chiffres représentés sur les images de img_test - _dimensions : (10000,)_ (NB : le ième label correspond au label de la ième image)  

Chaque image est représentée par un tableau de 28x28x1 valeurs (28 lignes x 28 colonnes x 1 couleur) comprises entre 0 et 255 (les pixels).  
Chaque valeur représente le niveau de gris d'un pixel (0 = noir, 255 = blanc)  

On peut le vérifier avec le code suivant :

In [ ]:
n_train = len(img_train) # nombre d'images pour l'entraînement
n_test = len(img_test) # nombre d'images pour le test

print("Nombre d'images dans l'échantillon d'entraînement : {}".format(n_train))
print("Nombre d'images dans l'échantillon de test : {}".format(n_test))
print()
print("Dimensions de img_train : {}".format(img_train.shape))
print("Dimensions de lbl_train : {}".format(lbl_train.shape))
print("Dimensions de img_test : {}".format(img_test.shape))
print("Dimensions de lbl_test : {}".format(lbl_test.shape))

In [ ]:
print("Dimensions d'une image : {}".format(img_train[0].shape))
print("Exemple de label d'une image : {}".format(lbl_train[0]))
print("Exemple de représentation d'une image : \n{}".format(np.matrix(img_train[0])))

On crée une fonction *print_img* pour afficher plus visuellement les images avec lesquelles nous travaillerons.

In [ ]:
def print_img(image, title = "", ax = None):
    if ax is None:
        f, ax = plt.subplots(1,1)
    ax.imshow(image.reshape(image.shape[0], image.shape[1]), cmap='gray')
    ax.set_title(title)
    if ax is None:
        plt.show()

In [ ]:
# on teste la fonction print_img sur une image de la base de données d'entraînement
idx = 1 # index de l'image à afficher

print_img(image = img_train[idx], \
          title = "Label : {}".format(lbl_train[idx]))

### <font color="indigo">Q1.1 : Que représentent les valeurs suivantes ?</font>
- ### <font color="indigo"> img_train[8764, 10, 20, 0]</font>
- ### <font color="indigo"> img_train[0, 0, 0, 0]</font>
- ### <font color="indigo">img_test[5487, 20, 15, 0]</font>
- ### <font color="indigo">img_test[11861, 8, 8, 0]</font>

---

### <font color="indigo">Q1.2 : Les données utilisées pour entraîner et tester l'IA sont cruciales pour créer une IA de bonne qualité. Quelle(s) analyse(s) peut-on faire sur les données d'entraînement et de test pour s'assurer que l'IA sera de bonne qualité et anticiper ses limites ? </font>

In [ ]:
# %load ../Solutions/Q12-Part1.py

In [ ]:
# %load ../Solutions/Q12-Part2.py

---

### <font color="indigo">Q1.3 : Au vu des résultats précédents, quelles limites de l'IA peut-on anticiper ? </font>

---

# <font color="gray">Partie 2 - Construction d'un réseau de neurones artificiel</font>

Avant de définir les réseaux de neurones artificiels, introduisons ce qu'est un __neurone__.

Un neurone est un "objet" informatique dont l'objectif est de __prédire un nombre $Y$ à partir de paramètres (ou prédicteurs) qu'on lui donne en entrée $I_{1}, ..., I_{N}$__. 

Il est composé de deux éléments :
- Ses __coefficients__ $W_{1}, ..., W_{N}$ qui mesureront l'importance relative des prédicteurs $I_{1}, ..., I_{N}$ sur la prédiction à réaliser. Le neurone apprendra tout seul à ajuster ses coefficients grâce à la base de données à partir de laquelle on l'entraînera. 
- Sa __fonction d'activation__ (ou threshold) qui doit être une fonction non linéaire et qui permet essentiellement au neurone de résoudre des problèmes non linéaires.  
_Exemples de fonctions d'activation connues : Sigmoïde, ReLu, SoftMax, ..._  

Pour l'aider à adapter ses coefficients, on definit 3 éléments supplémentaires, qui sont communs à tous les algorithmes de Machine Learning supervisés :
- Une __fonction de coût__ (loss function en anglais) qui permettra de mesurer à quel point les prédictions du neurone sont conformes à la réalité et qu'il s'agira de minimiser.  
*Exemples de fonctions de coût connues : categorical_crossentropy, binary_crossentropy, mean_squared_error, mean_absolute_error, ...*
- Une __fonction d'optimisation__ (optimizer en anglais) qui est un algorithme permettant d'adapter les coefficients du neurone $W_{1}, ..., W_{N}$ pour converger vers la solution qui minimise la fonction de coût.  
_Exemples de fonctions d'optimisation connues : Adam, AdaDelta, SGD, ..._
- Un __pas d'apprentissage__ (learning rate en anglais) qui permettra à la fonction d'optimisation de converger plus ou moins vite vers la solution qui minimise la fonction de coût (avec le risque si elle apprend trop vite de ne pas réussir à trouver la solution).

Comment tout cela fonctionne en pratique ?  
- Le neurone reçoit en entrée des paramètres $I_{1}, ..., I_{N}$ qui correspondent aux __prédicteurs__
- Il effectue la combinaison linéaire des paramètres en entrée avec ses __coefficients__ $W_{1}, ..., W_{N}$ : $ W_{1}I_{1} + ... + W_{1}I_{N}$
- Il applique à cette combinaison linéaire sa __fonction d'activation__ pour "casser" la linéarité et retourne un chiffre qui correspond à sa prédiction $\widehat{Y} = \sigma(W_{1}I_{1} + ... + W_{1}I_{N})$ (qu'il choisit au hasard au début)
- Il compare sa prédiction $\widehat{Y}$ avec la valeur attendue $Y$ dont il mesure la distance grâce à sa __fonction de coût__ $C(\widehat{Y} - Y)$
- En fonction de la distance entre sa prédiction et la valeur attendue, de sa __fonction d'optimisation__ et de son __pas d'apprentissage__, il ajuste ses __coefficients__ $W_{1}, ..., W_{N}$.
- Il réitère l'ensemble des étapes ci-dessus sur chaque donnée d'entraînement, par __batch__ de données d'entraînement pour accélérer les calculs
- Il réitère l'ensemble des étapes ci-dessus sur toute la base de données d'entraînement autant de fois que souhaité par l'utilisateur. on parle de nombre d'__epochs__

| *Représentation d'un neurone* |
|:--:|
|![](../Resources/images/Neurone.jpg)|

Un seul neurone n'est souvent pas suffisant pour obtenir de bons résultats car il ne calcule des coefficient que sur chaque paramètre d'entrée _pris individuellement_.   

On construit donc une architecture de neurones appelée __réseau de neurones artificiel__ qui va multiplier les possibilités de combinaisons des paramètres d'entrée. Un réseau de neurones artificiel est composé d'une ou plusieurs couche(s) de neurones, chacune comprenant plusieurs neurones. Chaque neurone peut alors envoyer son résultat à d'autres neurones.  

Par souci de cohérence, chaque neurone d'une même couche aura __la même fonction d'activation__. 

On applique alors à cette architecture __une fonction de coût, fonctions d'optimisation et un pas d'apprentissage__, qui permettront à chacun des neurones de la structure d'adapter leurs coefficients.

| *Exemple de réseau de neurones à deux couches* |
|:--:|
|![](../Resources/images/Réseau de neurones.png)|

Cette architecture, un peu "bourrine", est toutefois redoutablement efficace, comme nous allons le voir !

Pour commencer, on importe __keras__, qui est une librairie facile à utiliser pour créer des réseaux de neurones et qui repose sur __tensorflow__, librairie développée par Google pour faire du Deep Learning

In [ ]:
import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

### <font color="indigo">Q2.1 : Pour notre problème, quels paramètres devra prendre notre réseau de neurones en entrée et combien y en aura-t-il ?</font>

---

### <font color="indigo">Q2.2 : Combien de neurones doit-il y avoir dans la dernière couche ? Quelle sera la forme de la sortie du réseau de neurones ?</font>

---

### <font color="indigo">Q2.3 : Comment doit-on modifier les labels des images pour entraîner le réseau correctement ?</font>

In [ ]:
# %load ../Solutions/Q23.py

---

### <font color="indigo">Q2.4 : Comment interpréter le résultat que le réseau donnera ?</font>

In [ ]:
# %load ../Solutions/Q24.py

---

### <font color="indigo">Q2.5 : Pour que le réseau apprenne correctement, il est préférable que les données soient centrées autour de 0 et qu'elles aient une déviation standard d'environ 1. Pourquoi ? </font>

In [ ]:
# on centre les données sur zéros et on définit la déviation standard à 1
norm = ImageDataGenerator(featurewise_center= True, \
                          featurewise_std_normalization= True)

norm.fit(img_train)
norm_img_train = norm.flow(x= img_train, 
                           batch_size= n_train, 
                           shuffle= False).next()

norm_img_test = norm.flow(x= img_test,
                          batch_size= n_test,
                          shuffle= False).next()

# on vérifie le résultat
print("Moyenne de l'échantillon d'entraînement : {:.2}".format(norm_img_train.mean()))
print("Déviation standard de l'échantillon d'entraînement : {:.2}".format(norm_img_train.std()))

---

Nous sommes maintenant prêts pour créer notre réseau de neurones. Nous allons commencer par un réseau de neurones ayant les caractéristiques suivantes : 
- l'__architecture__ du réseau sera composée de 2 couches de neurones, constituées de 50 neurones pour la première, et de 10 neurones pour la deuxième
- la __fonction d'activation__ sera une fonction sigmoïde pour la première couche et un softmax pour la deuxième couche
- la __fonction de coût__ sera une fonction logarithmique
- la __fonction d'optimisation__ sera SGD
- le __pas d'apprentissage__ sera de 0.001    

Enfin, on entraînera le réseau de neurones par __batch de 32 images__ avec __10 epochs__.

In [ ]:
# On définit les paramètres-clés de notre réseau de neurones artificiels
input_shape = (28, 28, 1)                  # dimension des prédicteurs donnés en entrée du réseau
hidden_layer_size = 50                     # nombre de neurones dans la première couche couche
last_layer_size = 10                       # nombre de neurones dans la deuxième (dernière) couche
activation_function = 'sigmoid'            # fonction d'activation de la première couche
optimizer = optimizers.SGD                 # fonction d'optimisation
learning_rate = 0.001                      # pas d'apprentissage
loss_function = 'categorical_crossentropy' # fonction de coût
batch_size = 32                            # taille des batch d'entraînement
nb_epochs = 10                             # nombre d'itérations sur les données d'entraînement

In [ ]:
# On crée notre réseau de neurones artificiels simplement en utilisant l'API de Keras

ann = Sequential() # initialisation du réseau de neurones artificiels (artificial neural network)

ann.add(Flatten(input_shape = input_shape)) # on transforme les images en une liste de 28x28=784 données

ann.add(Dense(hidden_layer_size)) # première couche de 'hidden_layer_size' neurones
ann.add(Activation(activation_function)) # fonction d'activation de la première couche

ann.add(Dense(last_layer_size)) # deuxième (dernière) couche de 'last_layer_size' neurones
ann.add(Activation('softmax')) # fonction d'activation de la deuxième (dernière) couche

ann.compile(optimizer=optimizer(lr=learning_rate), # compilation du réseau de neurones artificiels
            loss=loss_function,   # - la fonction de coût à minimiser
            metrics=['accuracy']) # - l'indicateur à afficher

In [ ]:
# Entraînement du réseau de neurones avec les 60000 images de l'échantillon d'entraînement
# centrées autour de 0 et standardisées, avec les labels vectorisés

history = ann.fit(norm_img_train, vec_lbl_train, epochs=nb_epochs, batch_size=batch_size, \
                  validation_data=(norm_img_test, vec_lbl_test), verbose=1)

Différents indicateurs sont calculés pour chaque epochs :
- __loss__ : valeur de la fonction de coût sur l'échantillon d'entraînement après l'epoch (doit être la plus petite possible)
- __acc__ (pour __accuracy__) : précision de l'algorithme sur l'échantillon d'entraînement (qui correspond au nombre de prédictions correctes sur le nombre d'images d'entraînement)
- __val_loss__ (pour __validation_accuracy__) : valeur de la fonction de coût sur l'échantillon de test
- __val_acc__ (pour __validation_accuracy__) : précision de l'algorithme sur l'échantillon de test (qui correspond au nombre de prédictions correctes sur le nombre d'images de test)


### <font color="indigo">Q2.6 : Sur quels paramètres peut-on jouer pour tenter d'améliorer la précision du modèle ? Essayez d'améliorer le modèle sans toucher les paramètres *batch_size* et *nb_epochs*.</font>

---

On peut tracer l'historique des précisions sur les échantillons d'entraînement et de test après chaque epoch

In [ ]:
def plot_accuracy(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title("Précision de l'algorithme")
    plt.ylabel('Précision')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
plot_accuracy(history)

### <font color="indigo">Q2.7 : Que remarque-t-on ? (optionnel) Comment améliorer le résultat ?</font>

In [ ]:
# %load ../Solutions/Q27.py

---

Affichons quelques résultats pour vérifier que la plupart des prédictions sont correctes.

In [ ]:
def predict(images, nn, norm):
# Evalue la prédiction du réseau de neurones 'nn'
# sur les images 'images' (dimensions (,28,28,1)),
# normées avec le générateur 'norm
    norm_img = norm.flow(x=images, batch_size=len(images), shuffle=False).next()
    nn_outputs = nn.predict(norm_img)
    prediction = [nn_output.argmax() for nn_output in nn_outputs]
    confidence = [max(nn_output) for nn_output in nn_outputs]
    return prediction, confidence

In [ ]:
prediction, confidence = predict(images = img_test, nn = ann, norm = norm)

random_idx = np.random.choice(range(0,n_test), 100) # on choisit 100 images de test au hasard

# On affiche les images et les prédictions associées
f, axis = plt.subplots(10,10, figsize=(18,25))
for i,ax in zip(random_idx, axis.flatten()):
    print_img(image = img_test[i], \
              title = "Label : {}\nPrédiction : {}\nConf : {:.1%}".format(lbl_test[i], prediction[i], confidence[i]),
              ax = ax)
    ax.set_axis_off()
plt.show()

---

### <font color="indigo">Q2.8 : Quelles sont les principales limites d'un réseau de neurones artificiel ?</font>

---

# <font color="gray">Partie 3 - Construction d'un réseau de neurones convolutionnel</font>

Un __réseau de neurones convolutionnel__, utilisé pour la première fois par Yann LeCun et ses équipes (sur ce même problème de reconnaissance de chiffres manuscrits) est particulièrement adapté à la reconnaissance d'images. Il est  composé de deux parties :
- __Feature learning__ : cette partie permet l'apprentissage des caractéristiques pertinentes de l'image (contours, contrastes, formes, ...).
- __Classification__ : cette partie constituée d'un réseau de neurones artificiel, tel que décrit dans la dernière partie, permet d'apprendre à prédire ce que l'image représente, mais plus cette fois-ci à partir des images brutes, mais à partir des images filtrées dans la phase de feature learning.  

La phase de __feature learning__ comprend deux étapes, qui peuvent être répétées autant de fois que nécessaire :
- __Convolution__ : apprentissage et application de filtres pemettant de mettre en avant les caractéristiques de l'image (contours, contrastes, formes, ...). Ces filtres sont représentés par des matrices de taille précisée par l'utilisateur et dont les coefficients vont être appris par l'algorithme, de la même manière que les coefficients d'un réseau de neurones artificiel sont appris par l'algorithme construit dans la partie 2.  
Les filtres sont appliqués sur une image comme illustré sur le schéma ci-dessous :  

$$Filtre : \begin{pmatrix} 0 & 1 & 2 \\ 2 & 2 & 0 \\ 0 & 1 & 2 \end{pmatrix}$$


| *Convolution d'une image 5x5 par le filtre 3x3 indiqué ci-dessus* |
|:--:|
|![](../Resources/images/Convolution.gif)|

- __Pooling__ : réduction de la dimension de l'image (pour accélérer les calculs) en appliquant le Max ou la Moyenne des pixels de chaque partie de l'image.  
Le pooling s'effectue sur une image comme illustré sur le schéma ci-dessous (ici MaxPooling 2x2 avec un pas de 2)

| *Exemple de MaxPooling sur une image 4x4* |
|:--:|
|![](../Resources/images/Pooling.jpg)|

Une fois la phase de feature learning réalisée, les pixels de l'image ainsi traitée sont envoyés dans le réseau de neurones artificiels 

| *Exemple d'architecture d'un réseau de neurones convolutionnel* |
|:--:|
|![](../Resources/images/cnn.PNG)|

In [ ]:
# On importe les fonctions dont on aura besoin pour l'étape de feature learning
from keras.layers import Conv2D, MaxPooling2D

### <font color="indigo">Q3.1 : Effectuer la convolution de l'image : $$\begin{pmatrix} 2 & 3 & 0 & 0 \\ -1 & 1 & 3 & 1 \\ 0 & 1 & 2 & -1 \\ -1 & 0 & 0 & 1 \end{pmatrix}$$ par le filtre : $$\begin{pmatrix} 1 & -1 \\ -2 & 0 \end{pmatrix}$$  et appliquer un MaxPooling 2x2 avec un pas de 1 au résultat.</font>

In [ ]:
image = np.expand_dims(np.array([
        [ 2,  3,  0,  0],
        [-1,  1,  3,  1],
        [ 0,  1,  2, -1],
        [-1,  0,  0,  1]]), -1)

kernel = np.array([
        [ 1., -1.],
        [-2.,  0.],
    ])

In [ ]:
# %load ../Solutions/Q31.py

---

Affichons un exemple d'application de filtres à une image de notre base de données d'entraînement

In [ ]:
# quelques filtres connus
identity_kernel = np.array([
        [ 0.0,  0.0,  0.0],
        [ 0.0,  1.0,  0.0],
        [ 0.0,  0.0,  0.0]])

outline_kernel = np.array([
        [-1.0, -1.0, -1.0],
        [-1.0,  8.0, -1.0],
        [-1.0, -1.0, -1.0]])

blur_kernel = np.array([
        [ 1.0,  1.0,  1.0],
        [ 1.0,  1.0,  1.0],
        [ 1.0,  1.0,  1.0]])

horizontal_kernel = np.array([
        [ 0.0,  1.0,  0.0],
        [ 0.0,  -1.0,  0.0],
        [ 0.0,  0.0,  0.0]])

vertical_kernel = np.array([
        [ 0.0,  0.0,  0.0],
        [ 1.0,  -1.0,  0.0],
        [ 0.0,  0.0,  0.0]])

In [ ]:
img = img_train[1000]
maxpooling = False

f, axis = plt.subplots(2,3, figsize=(8,5))
print_img(img, title = "Image originale", ax = axis[0,0])
print_img(conv(img, identity_kernel, maxpooling), title = "Identity kernel", ax = axis[0,1])
print_img(conv(img, outline_kernel, maxpooling), title = "Outline kernel", ax = axis[0,2])
print_img(conv(img, blur_kernel, maxpooling), title = "Blur kernel", ax = axis[1,0])
print_img(conv(img, horizontal_kernel, maxpooling), title = "Horizontal kernel", ax = axis[1,1])
print_img(conv(img, vertical_kernel, maxpooling), title = "Vertical kernel", ax = axis[1,2])

for ax in axis.flatten(): ax.set_axis_off()

plt.show()

Nous allons maintenant créer notre réseau de neurones convolutionnel. Il sera composé de :
- Feature learning : 2 couches de Convolution apprenant respectivement 64 et 128 filtres 3x3, avec une fonction d'activation relu + MaxPooling 2x2 (avec un pas de 2)
- Classification : même architecture que le réseau de neurones artificiel mis au point dans la partie 2

Les fonctions de coût sera identique à celle de la partie 2, la fonction d'optimisation et le pas d'apprentissage seront identique à la partie 2.

Enfin, le réseau de neurones convolutionnel sera entraîné par batch de 32 images, sur seulement 3 epochs.

In [ ]:
# On définit les paramètres-clés de notre réseau de neurones convolutionnel
input_shape = (28, 28, 1)                  # dimension des prédicteurs donnés en entrée du réseau
conv_1_layer_size = 64                     # nombre de filtres de la première couche de convolution
conv_2_layer_size = 128                    # nombre de filtres de la deuxième couche de convolution
filter_size = (3,3)                        # dimension des filtres
pool_size = (2,2)                          # dimension de la fenêtre de MaxPooling
pool_stride = 2                            # pas du MaxPooling
optimizer = optimizers.adadelta            # fonction d'optimisation
learning_rate = 1                          # pas d'apprentissage
loss_function = 'categorical_crossentropy' # fonction de coût
batch_size = 32                            # taille des batch d'entraînement
nb_epochs = 3                              # nombre d'itérations sur les données d'entraînement

In [ ]:
# Initialisation
cnn = Sequential()

# Création des couches de feature learning
cnn.add(Conv2D(conv_1_layer_size,                # Première convolution
               kernel_size = filter_size,        
               activation = activation_function, 
               input_shape = input_shape))       

cnn.add(MaxPooling2D(pool_size = pool_size,      # Premier MaxPooling
                     strides = pool_stride))   

cnn.add(Conv2D(conv_2_layer_size,                # Deuxième convolution
               kernel_size = filter_size,
               activation = activation_function)) 

cnn.add(MaxPooling2D(pool_size = pool_size,      # Deuxième MaxPooling
                     strides = pool_stride))   

# Création des couches d'apprentissage (réseau de neurones artificiel)
### Copier ici la structure du réseau de neurones artificiel utilisé dans la partie 2
###
###

# Compilation
cnn.compile(optimizer=optimizer(lr = learning_rate),
              loss=loss_function,
              metrics=['accuracy'])         

### <font color="indigo">Q3.2 : Combien de paramètres y a-t-il en entrée du réseau de neurones artificiel ?</font>

---

On entraîne le réseau de neurones convolutionnel.

In [ ]:
cnn.fit(norm_img_train, vec_lbl_train, batch_size=batch_size, epochs = nb_epochs, \
          validation_data=(norm_img_test, vec_lbl_test), verbose=1)

On affiche quelques résultats.

In [ ]:
prediction, confidence = predict(images = img_test, nn = cnn, norm = norm)

random_idx = np.random.choice(range(0,n_test), 100) # on choisit 100 images de test au hasard

f, axis = plt.subplots(10,10, figsize=(18,25))
for i,ax in zip(random_idx, axis.flatten()):
    print_img(image = img_test[i], \
              title = "Label : {}\nPrédiction : {}\nConf : {:.1%}".format(lbl_test[i], prediction[i], confidence[i]),
              ax = ax)
    ax.set_axis_off()
plt.show()

Pour mieux comprendre comment fonctionnent les réseaux de neurones convolutionnels, on peut visualiser les sorties des 4 premières couches du réseau. C'est la dernière qui sera envoyée au réseau de neurones artificiel et qui remplace donc l'image originale.

In [ ]:
idx = 1000 # index de l'image sur laquelle visualiser les résultats
img = np.expand_dims(img_test[idx], 0)
norm_img = norm.flow(x=img, batch_size=len(img), shuffle=False).next()

def display_feature_map(layer, figsize = (20,5), nn = cnn, norm = norm):
    # fonction affichant toutes les images à la sortie de la couche 'layer' du réseau 'nn'
    print(nn.get_layer(index = layer).name)
    
    layer_cnn = Model(inputs=nn.inputs, outputs=nn.layers[layer].output)
    pred = layer_cnn.predict(norm_img)
    layer_output_size = pred.shape[-1]

    fig, axis = plt.subplots(int(layer_output_size/16), 16, figsize=figsize)
    for i,ax in zip(range(0, layer_output_size), axis.flatten()):
        print_img(image = pred[0,:,:,i], title="", ax = ax)
        ax.set_axis_off()
    plt.show()
    
# On affiche les images à la sortie des 4 premuères couches
for i in range(0,4):
    if i < 2 :
        figsize = (20,5)
    else:
        figsize = (20,10)
    display_feature_map(layer = i, figsize=figsize)

---

# <font color="gray">Partie 4 - Critiques des résultats</font>

On importe les librairies dont on aura besoin :
- __Lime__ : librairie très récente pour visualiser les facteurs d'influence les plus importants qui peuvent expliquent une décision d'une l'IA
- __Scikit-Learn__ : une librairie très utilisée en Machine Learning que nous utiliserons ici pour calculer des indicateurs de performance de l'algorithme
- __Scikit-Image__ : une librairie qui permet de travailler facilement avec les images

In [ ]:
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

from sklearn.metrics import confusion_matrix

from skimage.segmentation import mark_boundaries
from skimage.color import gray2rgb, rgb2gray, label2rgb

On charge les prédictions du réseau de neurones convolutionnel et on sépare les prédictions correctes des prédictions erronées

In [ ]:
# On charge les résultats du réseau de neurones convolutionel
prediction, confidence = predict(images = img_test, nn = cnn, norm = norm)

# On sépare les prédictions correctes des prédictions erronées
corr_idx = np.where(lbl_test - prediction == 0)[0]  # index des prédictions correctes
err_idx = np.where(lbl_test - prediction != 0)[0]   # index des prédictions erronées
print("Il y a {} erreurs de l'IA dans l'échantillon de test.".format(len(err_idx)))

On affiche les prédictions erronées (maximum 100 affichées) 

In [ ]:
f, axis = plt.subplots(10,10, figsize=(18,28))
for i,ax in zip(err_idx, axis.flatten()):
    print_img(image = img_test[i], \
              title = "Label : {}\nPrédiction : {}\nConf : {:.1%}\nIndex : {}"\
                      .format(lbl_test[i], prediction[i], confidence[i], i),
              ax = ax)
    ax.set_axis_off()
plt.show()

### <font color="indigo">Q4.1 : Quels constats peut-on faire ? Quelles analyses peut-on faire pour mieux caractériser les erreurs de l'IA ?</font>

In [ ]:
# %load ../Solutions/Q41-Hint.py

In [ ]:
# %load ../Solutions/Q41-Part1.py

In [ ]:
# %load ../Solutions/Q41-Part2.py

### <font color="indigo">Q4.2 : Sans toucher à l'algorithme, que peut-on faire pour minimiser les erreurs de l'IA ?</font>

In [ ]:
# %load ../Solutions/Q42-Hint.py

In [ ]:
# %load ../Solutions/Q42.py

---

On va maintenant essayer de comprendre les prédictions erronées de l'algorithme

In [ ]:
# On construit les objets qui permettront d'afficher les pixels les plus importants pour l'IA
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)

def predict_for_lime(img_rgb, nn = cnn, norm = norm):
    images = np.expand_dims((rgb2gray(img_rgb)*255).astype('uint8'),3)
    norm_img = norm.flow(x=images, batch_size=len(images), shuffle=False).next()
    prediction = nn.predict(norm_img)

    return prediction

In [ ]:
idx = 1000 # index de l'image à analyser

img = img_test[idx]
img_rgb = gray2rgb(img.reshape(28,28)).astype('uint8')

explanation = explainer.explain_instance(img_rgb, 
                                         classifier_fn = predict_for_lime, 
                                         top_labels=10, hide_color=0, num_samples=10000, segmentation_fn=segmenter)

fig, axis = plt.subplots(2,5, figsize = (20,8))
for i, ax in enumerate(axis.flatten()):
    temp, mask = explanation.get_image_and_mask(i, positive_only=True, num_features=10, hide_rest=False, min_weight = 0.)
    ax.imshow(label2rgb(mask,image=temp, bg_label = 0, alpha=0.6, colors=['green']))
    ax.set_title("Label : " + str(lbl_test[idx]) + \
                 "\nPrédiction : "+ str(i))
    ax.axis('off')
plt.show()

### <font color="indigo">Q4.3 : A partir des intuitions développées dans les partie 1 et des résultats précédents de LIME, comment peut-on modifier les images pour tromper l'algorithme ?</font>

In [ ]:
idx = 1000
img = np.array(img_test[idx])

In [ ]:
# %load ../Solutions/Q43-Part1.py

In [ ]:
# %load ../Solutions/Q43-Part2.py

In [ ]:
# %load ../Solutions/Q43-Part3.py

### <font color="indigo">Q4.4 : Comment améliorer la robustesse de l'algorithme ?</font>

In [ ]:
# %load ../Solutions/Q44.py

In [ ]:
#datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, rotation_range = 25, width_shift_range = 0.2, height_shift_range=0.2, zoom_range=0.2)
#datagen.fit(img_train)
#iterator = datagen.flow(img_train, vec_lbl_train, batch_size = 10)
#norm_img_test_aug = datagen.flow(img_test, shuffle = False, batch_size = 10000).next()
#cnn.fit_generator(iterator, epochs=4, steps_per_epoch=60000, validation_data=(norm_img_test_aug, vec_lbl_test))

---

# <font color="gray">Partie 5 - Utilisation d'un réseau de neurones convolutionnel puissant</font>

Comme on a pu le constater, nos ordinateurs peinent déjà à construire un réseau de neurones convolutionnel "simple" sur des images de 28x28 pixels.  
Heureusement, il existe sur le marché des réseaux de neurones autrement plus "profonds" pré-entraînés sur des millions d'images.  
C'est notamment le cas de l'algorithme ResNet dont les architectures sont présentés ci-dessous.

| *Architecture des réseaux ResNet* |
|:--:|
|![](../Resources/images/ResNet.png)|

Nous allons dans cette partie utiliser l'algorithme Mask-RCNN qui combine ResNet101 comme réseau de neurones convolutionnel et un puissant algorithme de segmentation d'images. Mask-RCNN est ainsi capable de détecter des objets sur une image.  

A l'heure actuelle, l'algorithme est capable de détecter les objets de la liste *class_names* ci-dessous.

In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

On importe les librairies que nous utiliserons dans cette partie :
- __Mask RCNN__ : pour utiliser l'algorithme Mask-RCNN
- __OpenCV__ : pour prendre des photos à l'aide de la webcam de l'ordinateur

In [ ]:
import sys 
sys.path.append('../') # pour se positionner dans le bon dossier pour importer mrcnn
import Resources.maskrcnn.mrcnn as mrcnn
from mrcnn import config, visualize
import mrcnn.model as modellib

import cv2

# On configure Mask-RCNN pour qu'il fonctionne sur notre ordinateur
class InferenceCocoConfig(config.Config):
    NAME = "inference_coco"
    NUM_CLASSES = 1 + 80  # COCO has 80 classes
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceCocoConfig()
model = modellib.MaskRCNN(mode="inference", model_dir='logs', config=config)

# On charge les coefficients du réseau de neurones sous-jacent à Mask-RCNN
coco_model_file = "../Resources/maskrcnn/mask_rcnn_coco.h5"
model.load_weights(coco_model_file, by_name=True)

Pour tester l'algorithme, on va utiliser des photos directement prises avec la webcam de notre ordinateur.  
On définit donc la fonction *camera_grab* pour cela.

In [ ]:
def camera_grab(camera_id=0):
    camera = cv2.VideoCapture(camera_id)
    try:
        for i in range(10):
            snapshot_ok, image = camera.read()
        if snapshot_ok:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            print("WARNING: could not access camera")
    finally:
        camera.release()
    return image

On prend une photo avec notre webcam

In [ ]:
pic = camera_grab()
plt.imshow(pic)

On teste l'algorithme Mask-RCNN avec notre photo

In [ ]:
def detect(picture):
    results = model.detect([picture], verbose=0)
    r = results[0]
    visualize.display_instances(pic, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
    
detect(pic)

On peut aussi utiliser une image sauvegardée sur notre disque

In [ ]:
#import os
#os.getcwd() # pour afficher le chemin actuel

In [ ]:
pic_path = "/Users/hugofayolle/Documents/Python/Cours/Cours Accenture/Reconnaissance d'images/Resources/maskrcnn/images/"
pic_filename = '12283150_12d37e6389_z.jpg'

pic = plt.imread(pic_path+pic_filename)
detect(pic)

### <font color="indigo">Q5.1 : Comment ce type d'algorithme pré-entraîné peut-il être utilisé dans l'industrie ?</font>